In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import os
import csv
import re
from time import sleep

In [2]:
def site_login():
    """
    셀레니움을 통해 사이트에 로그인을 한다.
    """
    driver.get ("http://www.catcare.or.kr/index.php?act=dispMemberLoginForm")
    driver.find_element_by_id("uemail").send_keys("여기는아이디입니다.")
    driver.find_element_by_id ("upw").send_keys("여기는비번이들어가야합니다.")
    driver.find_element_by_css_selector("#xe_popup2704778 > div.popupcloser > button").click()
    driver.find_element_by_class_name("btn_submit").click()

In [3]:
def save_to_csv(data):
#최초 생성 시
    if os.path.exists("./rescue_cat.csv")==False:
        #헤더를 저장한다.
        with open ("./rescue_cat.csv", "w", encoding='utf-8-sig',newline="") as fp:
            writer=csv.writer(fp)
            writer.writerow(["time", "title", "location", "view"])
        
    #데이터 저장
    with open("./rescue_cat.csv", 'a', encoding='utf-8-sig',newline="") as fp2:
        writer2=csv.writer(fp2)
        writer2.writerow(data)

In [4]:
#셀레니움 실행
driver = webdriver.Chrome("./chromedriver.exe")
#로그인한다.
site_login()

#반복하며 가져오기 위한 변수들을 준비한다.

#게시판으로 이동
driver.get ("http://www.catcare.or.kr/index.php?mid=resofaq&category=313908")

#게시판의 마지막 페이지
last_page=int(driver.find_element_by_xpath("//*[@id=\"bd_170_0\"]/div/form/fieldset/a[11]").text)

#현재 페이지 카운터
cur_page=1

In [5]:
while True:    
    #현재 게시글의 작성시간 카운터
    start_time=9

    #현재 페이지의 지역 카운터
    location_count=18

    #현재 페이지의 조회수 카운터
    view_count=19
    
    #2 증가 0으로 초기화
    k=0
    
    #길냥이사랑 구조문의 게시판으로 넘어간다.
    driver.get ("http://www.catcare.or.kr/index.php?mid=resofaq&category=313908&page="+str(cur_page))
    sleep(1)
    
    #게시판의 총 게시글 수
    row_count = len(driver.find_elements_by_xpath("//*[@id=\"bd_170_0\"]/div/table/tbody/tr"))
    
    #게시판의 총 공지글 수
    notice_count=len(driver.find_elements_by_xpath("//*[contains(@class,'notice')]"))
    
    #반복문 범위
    iter_range=(row_count-notice_count)
    
    
    #게시판을 파싱한다.
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    #현재 게시판 페이지에 있는 제목을 모두 저장한다.
    title_whole=soup.findAll('a', attrs={'class': 'hx'})
    
    #현재 게시판 페이지에 있는 지역과 조회수를 저장한다.(지역은 18부터 2씩 증가, 조회수는 19부터 2씩 증가)
    location_view_whole=soup.findAll('td', attrs={'class': 'm_no'})
    
    #현재 게시판 페이지에 있는 시간을 모두 저장한다.
    time_whole=soup.findAll('td', attrs={'class': 'time'})
           
    
    if cur_page<last_page:
        for i in range(iter_range):
            #1씩 증가한다.
            time=re.sub("\.", "-",time_whole[start_time+i].text)
            title=' '.join(re.sub("\W", " ", title_whole[i].text).split())


            #2씩 증가한다.
            location= ' '.join(re.sub("\W", " ", location_view_whole[location_count+k].text).split())
            view=location_view_whole[view_count+k].text

            #리스트에 담는다.
            data_list=[time, title, location, view]

            save_to_csv(data_list)

            #2씩 증가하는 변수 카운터
            k+=2
        
        cur_page+=1
        
    else:
        for i in range(iter_range):
            #1씩 증가한다.
            time=re.sub("\.", "-",time_whole[start_time+i].text)
            title=' '.join(re.sub("\W", " ", title_whole[i].text).split())

            #2씩 증가한다.
            location= ' '.join(re.sub("\W", " ", location_view_whole[location_count+k].text).split())
            view=location_view_whole[view_count+k].text
            
            #2씩 증가하는 변수 카운터
            k+=2
            
            #리스트에 담는다.
            data_list=[time, title, location, view]
                
            save_to_csv(data_list)
        break;
driver.close()